In [2]:
!uv pip install -q langchain==0.2.16 langchain-community==0.2.16 langchain-huggingface==0.0.3 langchain-text-splitters==0.2.4 chromadb==0.5.3 langchain-chroma==0.1.3 pandas ipywidgets

In [3]:
!uv pip install -q pyarrow unidecode

In [4]:
import pandas as pd
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain_chroma import Chroma
from langchain_text_splitters import CharacterTextSplitter
from tqdm import tqdm

In [5]:
import torch

In [6]:
import re

In [7]:
text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=3000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

In [ ]:
"""text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ".", " ", ""], 
    chunk_size=3000, #taille max d'un chunk = 3000
    chunk_overlap=200, #recouvrement de 200 caractères entre 2 chunks
    length_function=len, #taille du texte
    is_separator_regex=False, 
)"""


In [8]:
df = pd.read_parquet("data/echantillon_1000_hs_2024_TOC.parquet")
df = df.rename(columns={"numdossier_new":"numdossier"})
df= df.set_index("numdossier")

In [9]:
if (torch.cuda.is_available()):
    DEVICE="cuda"
else:
    DEVICE="cpu"
    
model_kwargs = {'device': "cuda"}
MODEL_NAME_EMBEDDER="BAAI/bge-m3"

embedder = HuggingFaceEmbeddings(
    model_name=MODEL_NAME_EMBEDDER, 
    model_kwargs=model_kwargs,
    show_progress=False
)

In [10]:
vector_store = Chroma(embedding_function=embedder,
                      persist_directory="./chroma_db_v0"  #répertoire pour stocker les données
)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [11]:
df.loc["T00724060165"]["accorddocx"]

"\n\nACCORD COLLECTIF D’ENTREPRISE\n\xa0\nEntre\xa0\n\xa0\nL’Association Mission Locale Nord Ardèche, dont le siège est situé 6 allée Nicolas Copernic- Tour H représenté par M.                   en sa qualité de Président.\n\xa0\nD’une part,\xa0\xa0\n\xa0\nEt\n\xa0\nLes délégués titulaires du CSE, représentant la majorité des suffrages exprimés lors des dernières élections professionnelles en date du 10/07/2023.\n\xa0\nD’autre part,\n\xa0\n\xa0\nPREAMBULE\n\xa0\nLes parties ont convenu de négocier un nouvel accord collectif d’entreprise suite à la dénonciation en date 06/12/2022 de l’accord collectif d’entreprise du 23/12/1999.\n\xa0\nLe présent accord prend en compte l’organisation actuelle en répondant aux besoins d’adaptation de la Mission Locale et à ses nécessités de fonctionnement.\n\xa0\nLe présent accord comporte notamment des dispositions concernant\xa0:\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\n\xa0\n\uf0b7\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0L’aménagement du temps de trava

In [12]:
def add_chunks_to_vectorstore(df, vector_store):
    for index, row in tqdm(df.iterrows(), total=len(df)):
        text=df.loc[index]["accorddocx"]
        texts = text_splitter.create_documents([text])
        i=0
        for t in texts:
            t.metadata["id"]=f"{index}_{i}"
            t.metadata["numdossier"]=f"{index}"
            vector_store.add_documents([t])
            i+=1


In [13]:
add_chunks_to_vectorstore(df,vector_store)

  0%|          | 0/1000 [00:00<?, ?it/s]/home/onyxia/work/.venv/lib/python3.13/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
  3%|▎         | 29/1000 [00:20<08:37,  1.88it/s]Created a chunk of size 7635, which is longer than the specified 3000
Created a chunk of size 12097, which is longer than the specified 3000
  5%|▍         | 46/1000 [00:35<09:52,  1.61it/s]Created a chunk of size 4556, which is longer than the specified 3000
Created a chunk of size 9016, which is longer than the specified 3000
Created a chunk of size 4032, which is longer than the specified 3000
Created a chunk of size 11177, which is longer than the specified 3000
Created a chunk of size 3949, which is longer than the specified 3000
Created a chunk of size 3526, which is longer than the specified 3000
Created a chunk of size 3369, which is l

In [14]:
!mc cp -r chroma_db_v0 s3/$VAULT_TOP_DIR/Rania/Accords

]11;?\

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...ma.sqlite3: 216.66 MiB / 216.66 MiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 191.87 MiB/s 1s